<a href="https://colab.research.google.com/github/KDiBSilva/Prediction-of-Product-Sales/blob/main/Prediction_of_Product_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Prediction of Product Sales**

Kristina DiBella Silva

##Project Overview

##Load and Inspect Data

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno

In [55]:
filename = '/content/drive/My Drive/Coding Dojo/Data/sales_predictions.csv'
sp_df = pd.read_csv(filename)

In [56]:
sp_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [57]:
sp_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


##Clean Data

**1) How many rows and columns?**

In [58]:
sp_df.shape

(8523, 12)

- There are 8523 rows and 12 columns. 

**2) What are the datatypes of each variable?**

In [59]:
sp_df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

**3) Are there duplicates? If so, drop any duplicates.**

In [60]:
sp_df.duplicated().sum()

0

- No duplicates found.

**4) Identify missing values.**

In [64]:
sp_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

- There are 1463 missing values in the Item_Weight column. 

- There are 2410 missing values in the Outlet_Size column.

In [65]:
sp_df.isna().sum() / len(sp_df) * 100

Item_Identifier               0.000000
Item_Weight                  17.165317
Item_Fat_Content              0.000000
Item_Visibility               0.000000
Item_Type                     0.000000
Item_MRP                      0.000000
Outlet_Identifier             0.000000
Outlet_Establishment_Year     0.000000
Outlet_Size                  28.276428
Outlet_Location_Type          0.000000
Outlet_Type                   0.000000
Item_Outlet_Sales             0.000000
dtype: float64

- Item_Weight is missing 17% of the values. 
- Outlet_Size is missing 28% of the values.

**5) Decide on how to address the missing values and do it! (This requires your judgement, so explain your choice).**

**Addressing the Outlet_Size missing values**
- 28% of the data for this column is missing, I  will be replacing the values with "Unknown".

In [66]:
sp_df['Outlet_Size'].fillna('Unknown', inplace= True)

**Addressing the missing Item_Weight values**
- I am going to check the mean of Item_Weight and compare it to the median. 

In [68]:
sp_df['Item_Weight'].mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of 0        9.300
1        5.920
2       17.500
3       19.200
4        8.930
         ...  
8518     6.865
8519     8.380
8520    10.600
8521     7.210
8522    14.800
Name: Item_Weight, Length: 8523, dtype: float64>

- Mean is 9.300

In [69]:
sp_df['Item_Weight'].median

<bound method NDFrame._add_numeric_operations.<locals>.median of 0        9.300
1        5.920
2       17.500
3       19.200
4        8.930
         ...  
8518     6.865
8519     8.380
8520    10.600
8521     7.210
8522    14.800
Name: Item_Weight, Length: 8523, dtype: float64>

- Median is 9.300

- There was no outlier as mean and median returned the same value 9.300.
- I will fill the missing values of Item_Weight with 9.300 using the average.

In [70]:
mean_itype = sp_df['Item_Weight'].mean
sp_df['Item_Weight'].fillna(mean_itype, inplace=True)


**6) Confirm that there are no missing values after addressing them.**

In [71]:
sp_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

- Item_Weight and Outlet_Size are no longer missing values.

**7) Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent).**

In [ ]:
sp_df.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

- Item_Fat_Content has five different values, however looking at the data dictionary there should only be two, Low Fat or Regular. 
- I will need to check what the other values are in order to replace them with the correct value. 

In [ ]:
sp_df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

- I will replace "LF" and "low fat" with "Low Fat"
- I will replace "reg" with "Regular"

In [72]:
sp_df['Item_Fat_Content'] = sp_df['Item_Fat_Content'].replace('LF','Low Fat')
sp_df['Item_Fat_Content'] = sp_df['Item_Fat_Content'].replace('low fat','Low Fat')
sp_df['Item_Fat_Content'] = sp_df['Item_Fat_Content'].replace('reg','Regular')

sp_df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

**8) For any numerical columns, obtain the summary statistics of each (min, max, mean).**

In [76]:
sp_df.describe().round(2)

,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8523.00,8523.00,8523.00,8523.00
mean,0.07,140.99,1997.83,2181.29
std,0.05,62.28,8.37,1706.50
min,0.00,31.29,1985.00,33.29
25%,0.03,93.83,1987.00,834.25
50%,0.05,143.01,1999.00,1794.33
75%,0.09,185.64,2004.00,3101.30
max,0.33,266.89,2009.00,13086.96


##Exploratory Data Analysis

##Explanatory Data Analysis